In [1]:
import sys; sys.path.append("..")
from policies import max_min_fairness, max_min_fairness_strategy_proof
from strategy_proofness_tests import test_strategy_proofness
import copy
import random
import time

import numpy as np
np.set_printoptions(precision=3, suppress=True)

In [2]:
def print_allocation_analysis(honest_throughputs, dishonest_throughputs,
                              strategy_proof_allocations, non_strategy_proof_allocations):
    print("=" * 100)
    print("Honest throughputs:", np.array(honest_throughputs))
    print("Dishonest throughputs:", np.array(dishonest_throughputs))
    print("-" * 100)

    for allocations, allocation_str in zip(
        [strategy_proof_allocations, non_strategy_proof_allocations],
        ["Strategy-proof", "Not strategy-proof"]):
        for allocation, throughput_str in zip(allocations,
                                              ["Honest throughputs", "Dishonest throughputs"]):

            print("%s (%s)\n" % (allocation_str, throughput_str))
            effective_throughputs = np.sum(np.multiply(np.array(honest_throughputs), allocation),
                                           axis=1)
            print("Allocation:", allocation)
            print("Effective throughputs:", effective_throughputs)
            print("-" * 100)
    print()
    print()

In [3]:
def harness(honest_throughputs, dishonest_throughputs,
            worker_types, cluster_spec):  
    strategy_proof_allocations = []
    non_strategy_proof_allocations = []
    for (throughputs, experiment_string) in zip(
        [honest_throughputs, dishonest_throughputs],
        ["Allocation computed using true throughputs",
         "Allocation computed using dishonest throughputs"]):

        throughputs = np.array(throughputs)
        unflattened_throughputs = {
            i: {worker_types[j]: throughputs[i][j] for j in range(len(worker_types))}
            for i in range(len(throughputs))
        }
        unflattened_strategy_proof_allocation, discount_factors, \
            unflattened_non_strategy_proof_allocation, runtime = \
                test_strategy_proofness(unflattened_throughputs, cluster_spec, verbose=False)
        strategy_proof_allocation = np.zeros((len(throughputs), len(cluster_spec)))
        non_strategy_proof_allocation = np.zeros((len(throughputs), len(cluster_spec)))
        for i in range(len(throughputs)):
            for j in range(len(worker_types)):
                strategy_proof_allocation[i][j] = \
                    unflattened_strategy_proof_allocation[i][worker_types[j]]
                non_strategy_proof_allocation[i][j] = \
                    unflattened_non_strategy_proof_allocation[i][worker_types[j]]

        strategy_proof_allocations.append(strategy_proof_allocation)
        non_strategy_proof_allocations.append(non_strategy_proof_allocation)

    effective_throughputs_honest_and_non_strategy_proof_allocation = \
        np.sum(np.multiply(honest_throughputs,
                           non_strategy_proof_allocations[0]), axis=1)
    effective_throughputs_dishonest_and_non_strategy_proof_allocation = \
        np.sum(np.multiply(honest_throughputs,
                           non_strategy_proof_allocations[1]), axis=1)
    effective_throughputs_honest_and_strategy_proof_allocation = \
        np.sum(np.multiply(honest_throughputs,
                           strategy_proof_allocations[0]), axis=1)
    effective_throughputs_dishonest_and_strategy_proof_allocation = \
        np.sum(np.multiply(honest_throughputs,
                           strategy_proof_allocations[1]), axis=1)

    if (effective_throughputs_honest_and_strategy_proof_allocation[0] /
           effective_throughputs_dishonest_and_strategy_proof_allocation[0] < 0.99):
        print("WARNING: Strategy-proof allocation is not actually strategy-proof!")
        print_allocation_analysis(honest_throughputs, dishonest_throughputs,
                                  strategy_proof_allocations, non_strategy_proof_allocations)
        return

    if (effective_throughputs_honest_and_non_strategy_proof_allocation[0] >=
        effective_throughputs_dishonest_and_non_strategy_proof_allocation[0]):
        return

    print("SUCCESS: Strategy-proof allocation is actually strategy-proof!")
    print_allocation_analysis(honest_throughputs, dishonest_throughputs,
                              strategy_proof_allocations, non_strategy_proof_allocations)

In [4]:
random.seed(42)
worker_types = ['v100', 'p100', 'k80']
cluster_spec = {'v100': 1, 'p100': 1, 'k80': 1}

for i in range(10):
    # throughputs specified in the above order.
    honest_throughputs = []
    for j in range(3):
        honest_throughputs.append(sorted([random.random(), random.random(), random.random()],
                                         reverse=True))
    dishonest_throughputs = copy.copy(honest_throughputs)
    dishonest_throughputs[0] = [random.random(), random.random(), random.random()]

    harness(honest_throughputs, dishonest_throughputs,
            worker_types, cluster_spec)

SUCCESS: Strategy-proof allocation is actually strategy-proof!
Honest throughputs: [[0.65  0.199 0.027]
 [0.589 0.545 0.22 ]
 [0.809 0.806 0.006]]
Dishonest throughputs: [[0.698 0.34  0.155]
 [0.589 0.545 0.22 ]
 [0.809 0.806 0.006]]
----------------------------------------------------------------------------------------------------
Strategy-proof (Honest throughputs)

Allocation: [[0.401 0.    0.1  ]
 [0.154 0.039 0.575]
 [0.    0.437 0.023]]
Effective throughputs: [0.263 0.238 0.353]
----------------------------------------------------------------------------------------------------
Strategy-proof (Dishonest throughputs)

Allocation: [[0.387 0.    0.203]
 [0.252 0.    0.48 ]
 [0.    0.467 0.   ]]
Effective throughputs: [0.257 0.254 0.376]
----------------------------------------------------------------------------------------------------
Not strategy-proof (Honest throughputs)

Allocation: [[0.475 0.    0.525]
 [0.525 0.261 0.214]
 [0.    0.739 0.261]]
Effective throughputs: [0.322 0

In [5]:
# case: job 1 slower on one resource than everyone else.
# Lying about having higher throughput on that resource will give higher ET with non-SP allocation
honest_throughputs = [[3, 1, 1],
                      [3, 2, 1],
                      [3, 2, 1]]
dishonest_throughputs = [[3, 1.1, 1],
                         [3, 2, 1],
                         [3, 2, 1]]
harness(honest_throughputs, dishonest_throughputs,
            worker_types, cluster_spec)

SUCCESS: Strategy-proof allocation is actually strategy-proof!
Honest throughputs: [[3 1 1]
 [3 2 1]
 [3 2 1]]
Dishonest throughputs: [[3.  1.1 1. ]
 [3.  2.  1. ]
 [3.  2.  1. ]]
----------------------------------------------------------------------------------------------------
Strategy-proof (Honest throughputs)

Allocation: [[0.32  0.    0.32 ]
 [0.164 0.339 0.164]
 [0.169 0.328 0.169]]
Effective throughputs: [1.28  1.333 1.333]
----------------------------------------------------------------------------------------------------
Strategy-proof (Dishonest throughputs)

Allocation: [[0.32  0.    0.32 ]
 [0.163 0.34  0.163]
 [0.17  0.327 0.17 ]]
Effective throughputs: [1.28  1.333 1.333]
----------------------------------------------------------------------------------------------------
Not strategy-proof (Honest throughputs)

Allocation: [[0.382 0.    0.618]
 [0.309 0.5   0.191]
 [0.309 0.5   0.191]]
Effective throughputs: [1.765 2.118 2.118]
------------------------------------------

In [6]:
honest_throughputs = [[3, 2, 0.9],
                      [3, 2, 1.1],
                      [3, 2, 1.1]]
dishonest_throughputs = [[3, 2, 1],
                         [3, 2, 1.1],
                         [3, 2, 1.1]]
harness(honest_throughputs, dishonest_throughputs,
            worker_types, cluster_spec)

SUCCESS: Strategy-proof allocation is actually strategy-proof!
Honest throughputs: [[3.  2.  0.9]
 [3.  2.  1.1]
 [3.  2.  1.1]]
Dishonest throughputs: [[3.  2.  1. ]
 [3.  2.  1.1]
 [3.  2.  1.1]]
----------------------------------------------------------------------------------------------------
Strategy-proof (Honest throughputs)

Allocation: [[0.022 0.639 0.   ]
 [0.32  0.01  0.331]
 [0.319 0.012 0.33 ]]
Effective throughputs: [1.345 1.345 1.345]
----------------------------------------------------------------------------------------------------
Strategy-proof (Dishonest throughputs)

Allocation: [[0.022 0.639 0.   ]
 [0.32  0.011 0.331]
 [0.32  0.011 0.331]]
Effective throughputs: [1.345 1.345 1.345]
----------------------------------------------------------------------------------------------------
Not strategy-proof (Honest throughputs)

Allocation: [[0.    0.989 0.011]
 [0.5   0.006 0.494]
 [0.5   0.006 0.494]]
Effective throughputs: [1.988 2.055 2.055]
------------------------

In [7]:
honest_throughputs = [[2.5, 2, 1],
                      [3, 2, 1],
                      [3, 2, 1]]
dishonest_throughputs = [[2.6, 2, 1],
                         [3, 2, 1],
                         [3, 2, 1]]
harness(honest_throughputs, dishonest_throughputs,
            worker_types, cluster_spec)

SUCCESS: Strategy-proof allocation is actually strategy-proof!
Honest throughputs: [[2.5 2.  1. ]
 [3.  2.  1. ]
 [3.  2.  1. ]]
Dishonest throughputs: [[2.6 2.  1. ]
 [3.  2.  1. ]
 [3.  2.  1. ]]
----------------------------------------------------------------------------------------------------
Strategy-proof (Honest throughputs)

Allocation: [[0.    0.64  0.   ]
 [0.333 0.    0.333]
 [0.333 0.    0.333]]
Effective throughputs: [1.28  1.333 1.333]
----------------------------------------------------------------------------------------------------
Strategy-proof (Dishonest throughputs)

Allocation: [[0.    0.64  0.   ]
 [0.333 0.    0.333]
 [0.333 0.    0.333]]
Effective throughputs: [1.28  1.333 1.333]
----------------------------------------------------------------------------------------------------
Not strategy-proof (Honest throughputs)

Allocation: [[0.    0.886 0.114]
 [0.5   0.057 0.443]
 [0.5   0.057 0.443]]
Effective throughputs: [1.886 2.057 2.057]
------------------------